# Deck checker

This little script is used to compare the code-generated deck to the version available on AnkiWeb. It is useful so the differences between the decks can be easily viewed. This ensures that changes to the deck are those expected.

## Prerequisits

 - Download the deck off anki-web
 - Run the corresponding python script (createJLPTDeck.py) and copy the `generated` folder to this directory 

Do this for the corresponding note type (`normal` or `extended`) 

The generated deck should be named `generated.anki2` and stored in the same directory as this notebook.
The deck off anki should be named `collection.anki2` and also stored in this directory.

`apkg` files are just zip files. Unzip it and you should find the `.anki2` file within.

In [11]:
import sqlite3
import os

import pandas as pd

# import qgrid

In [13]:
cols = ["Expression", "English definition", "Reading", "Grammar", "Additional definitions", "jlpt"]

def prepare_deck(level, name):
    # level = e.g. "jlpt-n5"
    # returns dataframe of the level-corresponding deck from the web deck
    # name is the file name of the .anki2 file

    if not os.path.isfile(name):
        raise FileNotFoundError(f'{name} does not exist.')
    
    # open the sql db as a dataframe
    conn = sqlite3.connect(name)
    query = "SELECT * FROM notes"
    df_col = pd.read_sql_query(query,conn)
    
    df5 = df_col[df_col["tags"].str.contains(level)]
    fls = df5["flds"].apply( lambda x: x.split('\x1f'))

    df = pd.DataFrame()
    for i,c in enumerate(cols):
        df[c] = fls.str[i]
    df["jlpt"] = df5["tags"]
    df = df.sort_values('Expression')
    df = df.reset_index(drop=True)
    return df

In [4]:
def generated_deck(level):
    # opens the generated csv file as a dataframe
    # level = e.g. "jlpt-n5"
    df = pd.read_csv(f"generated/{level}normal.csv", header=None)
    # formality column not contained in web deck currently
    df.drop(df.columns[[5]], axis=1, inplace=True)
    df.columns = cols
    df = df.sort_values('Expression')
    df = df.reset_index(drop=True)
    df.fillna('', inplace=True)
    return df


In [5]:
def compare(df1, df2):
    # find rows not present in the other
    # returns: differences in both dfs
    #          rows in left df not in right
    #          rows in right df not in left
    df_all = df1.merge(df2.drop_duplicates(), on=['Expression'], 
                   how='left', indicator=True, suffixes=('', '_y'))
    df_all = df_all.drop(df_all.filter(regex='_y$').columns.tolist(),axis=1)
    df_left =  df_all[df_all['_merge'] == 'left_only']
    df_left = df_left.drop(columns="_merge")
    
    
    df_all = df1.merge(df2.drop_duplicates(), on=['Expression'], 
                   how='right', indicator=True, suffixes=('_x', ''))
    df_all = df_all.drop(df_all.filter(regex='_x$').columns.tolist(),axis=1)
    df_right =  df_all[df_all['_merge'] == 'right_only']
    df_right = df_right.drop(columns="_merge")
    
    df1 = df1.drop(index = df_left.index)
    df2 = df2.drop(index = df_right.index)
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    
    df_comp = df1.compare(df2)
    
    return (df_comp, df_left, df_right)

In [6]:
def show_diff(df1, df2):
    # show all changes between two dataframes, except jtlp tags
    df,_,_ = compare(df1, df2)
    df.drop(columns="jlpt")

In [7]:
def show_reading_diff(df1, df2):
    # table showing the different readings between 2 dataframes
    df_all,_,_ = compare(df1,df2)
    dgrid = df_all.drop(columns=['English definition', 'Grammar', "Additional definitions", "jlpt"])
    dgrid = dgrid.dropna(how="all")
    dgrid
    dgrid.columns = [' '.join(col).strip() for col in dgrid.columns.values]
    return dgrid
    qgrid.show_grid(dgrid)


In [7]:
def reading(web: pd.DataFrame, generated: pd.DataFrame):
    # take only the new readings, and and new rows
    # web and generated are dataframes
    # returns a dataframe ready for use with qgrid.show(return)

    df_fin = web.merge(generated, how="outer", on="Expression")
    df_fin["Reading_x"] = df_fin["Reading_y"]
    df_fin = df_fin.drop(df_fin.filter(regex='_y$').columns.tolist(),axis=1)
    df_fin.columns = df_fin.columns.str.rstrip("_x")
    return df_fin

In [14]:
# levels=["jlpt-n5", "jlpt-n4", "jlpt-n3", "jlpt-n2", "jlpt-n1", "common"]
for l in ["jlpt-n5"]:
    dfg = prepare_deck(l, 'generated.anki2')
    dfw = prepare_deck(l, 'collection.anki2')
    df_all, df_left, df_right = compare(dfw, dfg)
    df_r = reading(dfw, dfg)
    df_r.to_csv(f'generated/{l}-read.csv', encoding="utf-8", index=False, header=False)
df_all.columns = [' '.join(col).strip() for col in df_all.columns.values]
# qgrid.show_grid(df_all)


FileNotFoundError: generated.anki2 does not exist

In [23]:
df_left

,Expression,English definition,Reading,Grammar,Additional definitions,jlpt
646,鳥肉,"chicken meat, bird meat",鳥肉[とりにく],Noun,,jlpt-n5


In [18]:
qgrid.show_grid(show_reading_diff(dfw,dfg))


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
# only check different readings
dgrid = df_all.drop(columns=['English definition', 'Grammar', "Additional definitions", "jlpt"])
dgrid = dgrid.dropna(how="all")
dgrid

In [ ]:
dfg = generated_deck("jlpt-n5")
dfw = prepare_deck("jlpt-n5")
df_all, df_left, df_right = compare(dfw, dfg)
reading(dfw, dfg)

In [ ]:
qgrid.show_grid(show_reading_diff(dfw,dfg))

In [24]:
df_all, df_left, df_right = compare(dfw, dfg)
# present in online deck, not generated table
df_left
# present in generated table, not in online deck
df_right
df_all

NameError: name 'dd5' is not defined